<a href="https://colab.research.google.com/github/daleas0120/townhall_tutorial/blob/main/nb1_loss_landscape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to calculate a loss landscape

Ashley S. Dale

## Summary

1. Train a small model on a toy dataset
    - We will use the [Glass Identification Dataset](http://archive.ics.uci.edu/dataset/42/glass+identification) from UC Irvine Machine Learning Repository
2. Calculate the Hessian of the model
3. Calculate the eigenvalues and eigenvectors of the Hessian
4. Calculate the loss landscape for the model

In [ ]:
# install data repository
%pip install ucimlrepo --quiet

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy

# Data Processing
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Model training & Hessian Calculation
import torch
import torch.nn as nn
import torch.optim as optim

# Hessian Eigenvectors and Values
from numpy import linalg as LA


## Data Preprocessing

`Double click this cell to edit`

Add documentation to the notebook:

Write a paragraph that describes the logic in the next two code cells

In [ ]:
# load the dataset by running this cell
glass_identification = fetch_ucirepo(id=42)

# extract the refractive index and chemical formula information
raw_data = glass_identification.data.features
glass_kinds = glass_identification.data.targets

# show a Pandas Dataframe table of data in the notebook
# raw_data

In [ ]:
X, y = raw_data.iloc[:, 1:], raw_data.iloc[:, 0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


## Define & Train Model

`Double click this cell to edit`

Add documentation to the notebook:

Write a paragraph that describes the logic in the next two code cells

In [ ]:
class LinearRegressionModel(nn.Module):
  def __init__(self, input_dim, output_dim):
      super(LinearRegressionModel, self).__init__()
      self.linear = nn.Linear(input_dim, output_dim)

  def forward(self, x):
      out = self.linear(x)
      return out


In [ ]:
model = LinearRegressionModel(X_train.shape[1], 1)
print(model)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}')

# Evaluation
model.eval()
with torch.no_grad():
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).reshape(-1, 1)
    y_pred = model(X_test_tensor)

    test_loss = criterion(y_pred, torch.tensor(y_test.values, dtype=torch.float32).reshape(-1, 1))
    print(f'Test Loss: {test_loss.item():.6f}')

## Model Hessian Calculation

In [ ]:
# Clear the existing model gradients
model.zero_grad()

In [ ]:
# Calculate the first gradients
grads = torch.autograd.grad(loss, model.parameters(), create_graph=True, retain_graph=True)
print(*(g.shape for g in grads))

# Flatten the gradients into a single vector
grads_flat = torch.cat([g.contiguous().view(-1) for g in grads])
print(grads_flat.shape)

In [ ]:
# Create empty matrix to hold the gradient calculation
num_params = grads_flat.numel()
hessian = []

# TODO: fill in the blanks in the code
# for each gradient in the `grads_flat` vector
for g_idx in grads_flat:

    #TODO: get the gradient w.r.t. the gradient g_idx
    grad2 =

    #TODO: flatten the second derivative into a vector
    grad2_flat =

    #TODO: Add it to the hessian list
    hessian.append(grad2_flat)

hessian = torch.stack(hessian)

In [ ]:
# TODO: Visualize the Hessian Matrix by completing the plt.imshow() command
plt.imshow()
plt.plot()

## Get the Hessian Eigenvalues and Eigenvectors

In [ ]:
hessian = hessian.detach().numpy()

# Solve for the eigenvalues and eigenvectors
eigenvalues, eigenvectors = LA.eig(hessian)

# Format and sort the eigenvalues and eigenvectors for use
eigenvectors = [eigenvectors[:][i] for i in range(num_params)]
eigenvalues = list([i] for i in eigenvalues)

eigenvalues, eigenvectors = zip(*sorted(zip(eigenvalues, eigenvectors)))
eigenvalues, eigenvectors = list(eigenvalues), list(eigenvectors)

In [ ]:
# TODO: Plot the eigenvalues to check they are sorted; complete plt.plot() command
plt.plot()
plt.xlabel('ith Eigenvalue')
plt.ylabel('Eigenvalue')
plt.show()

In [ ]:
# Convert eigenvectors to torch vectors
eig1 = torch.tensor(eigenvectors[-1].reshape(1, -1), dtype=torch.float32)
eig2 = torch.tensor(eigenvectors[-2].reshape(1, -1), dtype=torch.float32)

## Calculate the Loss Landscape

In [ ]:
# Loss Landscape Hyperparameters
loss_landscape_model = copy.deepcopy(model)
n_steps = 100

# shift model origin to be the middle of the plot
loss_landscape_model.linear.weight.data = loss_landscape_model.linear.weight.data - (eig1[:, :-1]/2.) - (eig2[:, :-1]/2.)
loss_landscape_model.linear.bias.data = loss_landscape_model.linear.bias.data - (eig1[:, -1]/2.) - (eig2[:, -1]/2.)

# determine the update increment
eig1_step = eig1/n_steps
eig2_step = eig2/n_steps

loss_landscape = []

for _ in range(0, n_steps):
  row = []

  loss_landscape_model.linear.weight.data = loss_landscape_model.linear.weight.data + eig2_step[:, :-1]
  loss_landscape_model.linear.bias.data = loss_landscape_model.linear.bias.data + eig2_step[:, -1]

  for j in range(1, n_steps+1):
    #TODO: add eig1_step weight values to the model weight parameters
    loss_landscape_model.linear.weight.data =

    #TODO: add eig1_step bias values to the model bias parameters
    loss_landscape_model.linear.bias.data =

    #TODO: get predictions with the updated loss_landscape_model for the X_test_tensor data
    y_pred =

    #TODO: calculate the loss between the y_pred variable and the y_test_tensor variables
    loss =

    # save the loss to the row of the loss landscape
    row.append(loss.item())

  loss_landscape.append(row)

  # Rewind the model for the next row
  loss_landscape_model.linear.weight.data = loss_landscape_model.linear.weight.data - j*eig1_step[:, :-1]
  loss_landscape_model.linear.bias.data = loss_landscape_model.linear.bias.data - j*eig1_step[:, -1]

In [ ]:
#TODO: Plot the loss landscape
plt.imshow()
plt.colorbar()
plt.show()

# Open Questions

1. What happens if the model is trained for more epochs? (reset the notebook kernel before doing this)
1. If you change which eigenvectors are used for the calculation, what happens to the loss landscape?
1. Add some noise to the dataset using a Gaussian random variable; how does affect the loss landscape?
1. Can you reduce the model dimension so that there are no non-zero eigenvectors? (Hint: Consider removing redundant dataset features)